# The Data Catalogue vocabulary (DCAT)

DCAT is a RDF vocabulary for
describing any dataset. A dataset is a collection of data, published
or curated by a single source, and available for access or donwnload
in one or more formats. DCAT does not include support for semantic
relations to concepts, or specification of protocols and parameters
for datasets available through APIs

DCAT-AP is an extension of DCAT that combines DCAT with further
metadata that allows data to be more easily searchable and
discoverable, also supporting multiple languages.

Digi provides a standard for describing datasets and data cataloges
(DCAT-AP-NO)[ref] which is based on the DCAT-AP standard [ref]. 


# Simple Knowledge Organization System (SKOS)

SKOS is a lightweight vocabulary that formally represents knowledge
organization using RDF. DCAT combined with SKOS allows for datasets to
be linked to an application taxonomy (application vocabulary).


## Building a open traffic dataset vocabulary (OTD-vocabulary)
Now we have a standardized framework on which we can base the ODT
vocabulary. By building the taxonomy on the SKOS vocabulary allows
for extending a high-level ontology with domain specific ontologies.


## Automatic classification

The registration of new datasets are often performed by stakeholders
with little or no knowledge about the technical side of the
information organization and classification schemes in use. Yet it
should be reasonable simple for anyone to register new data. In this
section the strategies for automatic data classification is discussed
and results from studies are presented.

## Keyword based classification

By providing a framework where keywords can be entered together with
the datasets, it is possible to match the keywords to the standard
vocabulary for concepts. Predefined annotated keywords allows for a
direct match, while user-defined keywords can be matched to existing
vocabularies by using WordNet as a reference tool, where the semantic
similarity can be computed using a thesaurus-based metric such as path
similarity, Leacock-Chodorow similarity[ref] or Wu-Palmer Similarity.

## Standard (known) formats classification

Domain specific taxonomies developed to for known dataset structures
such as SIRI 2.0 Lite allows for correct classification of relevant
datasets.

## Inferred classification based on known schemas

Datasets that contain structured information with embedded meta-data
can to some degree be classified based on the interpretation of the
meta-data. For XML data with reference to a XML Schema Definition
(XSD), the named xml-schema elements can be matched with known
vocabularies. Other types of structured data, such as CSV-files often
include a column header field that can be matched the same way as the
XSD files. JSON/YAML-documents are structured and nested keyword-value
pairs. Here the keywords can be treated the same way as the formats
indicated above.

Steps:

  0. data parsing
  1. concept extraction
  2. concept matching
  3. linking


# Creating a new instance of the core ODT ontology
The ODT ontology is build on top of the Simple Knowledge Organization System (SKOS) for formal

# Adding a dataset with a set of keywords

When adding a new dataset, we can make use of the DCAT vocabulary and store it in JSON-LD format.

In [17]:
from rdflib import Graph, URIRef, BNode, Literal, Namespace
from rdflib.namespace import RDF, RDFS, OWL, DC, FOAF, XSD
from rdflib.plugins.sparql import prepareQuery

OTD = Namespace('http://www.quaat.com/ontologies#')
DCAT = Namespace('http://www.w3.org/ns/dcat#')
DCT = Namespace('http://purl.org/dc/terms/')
SKOS = Namespace('http://www.w3.org/2004/02/skos/core#')
OTDX = Namespace('http://www.quaat.com/ontology/OTDX#')
QEX = Namespace('http://www.quaat.com/extended_skos#')

g = Graph()
g.bind('foaf', FOAF)
g.bind('dct', DCT)
g.bind('dcat', DCAT)
g.bind('rdf', RDF)
g.bind('dc', DC)
g.bind('skos', SKOS)
g.bind('otd', OTD)
g.bind('qex', QEX)

g.parse('skos.rdf')
#g.parse('dcat-ap_1.1.rdf')
#g.parse('myschema.rdf')
thomas = BNode()
g.add((thomas, RDF.type, FOAF.Person))
g.add((thomas, FOAF.name, Literal("Thomas Hagelien")))

def add_node(name, parent, **kwargs):
    node = URIRef(OTD + name) 
    g.add((node, RDF.type, SKOS.Concept))
    g.add((node, SKOS.broader, parent))
    if ('prefLabel' in kwargs):
        for label in kwargs['prefLabel']:
            g.add((node, SKOS.prefLabel, label))
    if ('altLabel' in kwargs):
        for label in kwargs['altLabel']:
            g.add((node, SKOS.altLabel, label))
    g.add((node, SKOS.inScheme, OTDScheme))
    return node

OTDScheme = URIRef(OTD + 'OTDScheme') 
g.add((OTDScheme, RDF.type, SKOS.ConceptScheme))
g.add((OTDScheme, DC.title, Literal("Open Transport Datas Taxonomy")))
g.add((OTDScheme, DC.description, Literal("Taxonomy for the classification of Open Transport Data")))
g.add((OTDScheme, DC.creator, thomas))

Entity = URIRef(OTD + 'Entity')
g.add((Entity, SKOS.prefLabel, Literal("entity", lang="en")))
g.add((Entity, SKOS.prefLabel, Literal("entitet", lang="nb")))
g.add((Entity, SKOS.topConceptOf, OTDScheme)) # Add a top concept to allow for querying the taxonomy

Abstraction = add_node('Abstraction', 
                       Entity, 
                       prefLabel=[Literal("abstraction", lang="en"),
                                  Literal("abstraksjon", lang="nb")])
Communication = add_node('Communication', 
                       Abstraction, 
                       prefLabel=[Literal("communication", lang="en"),
                                  Literal("kommunikasjon", lang="nb")])


Agreement = URIRef(OTD + 'Agreement')
g.add((Agreement, RDF.type, SKOS.Concept))
g.add((Agreement, SKOS.broader, Communication))
g.add((Agreement, SKOS.prefLabel, Literal("agreement", lang="en")))
g.add((Agreement, SKOS.prefLabel, Literal("avtale", lang="nb")))
g.add((Agreement, SKOS.inScheme, OTDScheme))

Statistics = URIRef(OTD + 'Statistics')
g.add((Statistics, RDF.type, SKOS.Concept))
g.add((Statistics, SKOS.broader, Abstraction))
g.add((Statistics, SKOS.prefLabel, Literal("statistics", lang="en")))
g.add((Statistics, SKOS.prefLabel, Literal("statistikk", lang="nb")))
g.add((Statistics, SKOS.inScheme, OTDScheme))

APIDescription = URIRef(OTD + 'APIDescription')
g.add((APIDescription, RDF.type, SKOS.Concept))
g.add((APIDescription, SKOS.broader, Abstraction))
g.add((APIDescription, SKOS.prefLabel, Literal("API description", lang="en")))
g.add((APIDescription, SKOS.prefLabel, Literal("grensesnitt", lang="nb")))
g.add((APIDescription, SKOS.inScheme, OTDScheme))

TransportMode = URIRef(OTD + 'TransportMode')
g.add((TransportMode, RDF.type, SKOS.Concept))
g.add((TransportMode, SKOS.broader, Abstraction))
g.add((TransportMode, SKOS.prefLabel, Literal("transport mode", lang="en")))
g.add((TransportMode, SKOS.prefLabel, Literal("transporttype", lang="nb")))
g.add((TransportMode, SKOS.inScheme, OTDScheme))

Air = URIRef(OTD + 'Air')
g.add((Air, RDF.type, SKOS.Concept))
g.add((Air, SKOS.broader, TransportMode))
g.add((Air, SKOS.prefLabel, Literal("air", lang="en")))
g.add((Air, SKOS.prefLabel, Literal("luft", lang="nb")))
g.add((Air, SKOS.inScheme, OTDScheme))

Sea = URIRef(OTD + 'Sea')
g.add((Sea, RDF.type, SKOS.Concept))
g.add((Sea, SKOS.broader, TransportMode))
g.add((Sea, SKOS.prefLabel, Literal("sea", lang="en")))
g.add((Sea, SKOS.prefLabel, Literal("hav", lang="nb")))
g.add((Sea, SKOS.inScheme, OTDScheme))

Rail = URIRef(OTD + 'Rail')
g.add((Rail, RDF.type, SKOS.Concept))
g.add((Rail, SKOS.broader, TransportMode))
g.add((Rail, SKOS.prefLabel, Literal("rail", lang="en")))
g.add((Rail, SKOS.prefLabel, Literal("skinner", lang="nb")))
g.add((Rail, SKOS.inScheme, OTDScheme))

Cable = URIRef(OTD + 'Cable')
g.add((Cable, RDF.type, SKOS.Concept))
g.add((Cable, SKOS.broader, TransportMode))
g.add((Cable, SKOS.prefLabel, Literal("cable", lang="en")))
g.add((Cable, SKOS.prefLabel, Literal("kabel", lang="nb")))
g.add((Cable, SKOS.inScheme, OTDScheme))

Information = URIRef(OTD + 'Information')
g.add((Information, RDF.type, SKOS.Concept))
g.add((Information, SKOS.broader, Communication))
g.add((Information, SKOS.prefLabel, Literal("information", lang="en")))
g.add((Information, SKOS.prefLabel, Literal("informasjon", lang="nb")))
g.add((Information, SKOS.inScheme, OTDScheme))

GeographicalInformation = URIRef(OTD + 'GeographicalInformation')
g.add((GeographicalInformation, RDF.type, SKOS.Concept))
g.add((GeographicalInformation, SKOS.broader, Communication))
g.add((GeographicalInformation, SKOS.prefLabel, Literal("geographical information", lang="en")))
g.add((GeographicalInformation, SKOS.prefLabel, Literal("geografisk informasjon", lang="nb")))
g.add((GeographicalInformation, SKOS.altLabel, Literal("geoinformasjon", lang="nb")))
g.add((GeographicalInformation, SKOS.inScheme, OTDScheme))

Map = URIRef(OTD + 'Map')
g.add((Map, RDF.type, SKOS.Concept))
g.add((Map, SKOS.broader, GeographicalInformation))
g.add((Map, SKOS.prefLabel, Literal("map", lang="en")))
g.add((Map, SKOS.prefLabel, Literal("kart", lang="nb")))
g.add((Map, SKOS.inScheme, OTDScheme))

EnvironmentInformation = URIRef(OTD + 'EnvironmentInformation')
g.add((EnvironmentInformation, RDF.type, SKOS.Concept))
g.add((EnvironmentInformation, SKOS.broader, Information))
g.add((EnvironmentInformation, SKOS.prefLabel, Literal("environment information", lang="en")))
g.add((EnvironmentInformation, SKOS.prefLabel, Literal("miljøinformasjuton", lang="nb")))
g.add((EnvironmentInformation, SKOS.inScheme, OTDScheme))

EmissionInformation = URIRef(OTD + 'EmissionInformation')
g.add((EmissionInformation, RDF.type, SKOS.Concept))
g.add((EmissionInformation, SKOS.broader, EnvironmentInformation))
g.add((EmissionInformation, SKOS.prefLabel, Literal("emission information", lang="en")))
g.add((EmissionInformation, SKOS.altLabel, Literal("emission", lang="en")))
g.add((EmissionInformation, SKOS.prefLabel, Literal("utslipp", lang="nb")))
g.add((EmissionInformation, SKOS.altLabel, Literal("utslippsdata", lang="nb")))
g.add((EmissionInformation, SKOS.altLabel, Literal("discharge", lang="nb")))
g.add((EmissionInformation, SKOS.inScheme, OTDScheme))

MeteorologicalInformation = URIRef(OTD + 'MeteorologicalInformation')
g.add((MeteorologicalInformation, RDF.type, SKOS.Concept))
g.add((MeteorologicalInformation, SKOS.broader, EnvironmentInformation))
g.add((MeteorologicalInformation, SKOS.prefLabel, Literal("meteorological information", lang="en")))
#g.add((MeteorologicalInformation, SKOS.altLabel, Literal("weather condition", lang="en")))
g.add((MeteorologicalInformation, SKOS.prefLabel, Literal("meteorologisk informasjon", lang="nb")))
g.add((MeteorologicalInformation, SKOS.inScheme, OTDScheme))

HydrometeorologicalInformation = URIRef(OTD + 'HydrometeorologicalInformation')
g.add((HydrometeorologicalInformation, RDF.type, SKOS.Concept))
g.add((HydrometeorologicalInformation, SKOS.broader, EnvironmentInformation))
g.add((HydrometeorologicalInformation, SKOS.prefLabel, Literal("hydrometeorological information", lang="en")))
g.add((HydrometeorologicalInformation, SKOS.prefLabel, Literal("hydrometeorologisk informasjon", lang="nb")))
g.add((HydrometeorologicalInformation, SKOS.inScheme, OTDScheme))

AirQuality = URIRef(OTD + 'AirQuality')
g.add((AirQuality, RDF.type, SKOS.Concept))
g.add((AirQuality, SKOS.broader, EnvironmentInformation))
g.add((AirQuality, SKOS.prefLabel, Literal("air quality", lang="en")))
g.add((AirQuality, SKOS.prefLabel, Literal("luftkvalitet", lang="nb")))
g.add((AirQuality, SKOS.inScheme, OTDScheme))

TravelInformation = URIRef(OTD + 'TravelInformation')
g.add((TravelInformation, RDF.type, SKOS.Concept))
g.add((TravelInformation, SKOS.broader, Information))
g.add((TravelInformation, SKOS.prefLabel, Literal("travel information", lang="en")))
g.add((TravelInformation, SKOS.prefLabel, Literal("reiseinformation", lang="nb")))
g.add((TravelInformation, SKOS.inScheme, OTDScheme))

RealTimeInformation = URIRef(OTD + 'RealTime')
g.add((RealTimeInformation, RDF.type, SKOS.Concept))
g.add((RealTimeInformation, SKOS.broader, TravelInformation))
g.add((RealTimeInformation, SKOS.prefLabel, Literal("real time", lang="en")))
g.add((RealTimeInformation, SKOS.prefLabel, Literal("sanntid", lang="nb")))
g.add((RealTimeInformation, SKOS.inScheme, OTDScheme))

TravelPlan = URIRef(OTD + 'TravelPlan')
g.add((TravelPlan, RDF.type, SKOS.Concept))
g.add((TravelPlan, SKOS.broader, TravelInformation))
g.add((TravelPlan, SKOS.prefLabel, Literal("travel plan", lang="en")))
g.add((TravelPlan, SKOS.prefLabel, Literal("reiseplan", lang="nb")))
g.add((TravelPlan, SKOS.inScheme, OTDScheme))

RoutePlan = URIRef(OTD + 'RoutePlan')
g.add((RoutePlan, RDF.type, SKOS.Concept))
g.add((RoutePlan, SKOS.broader, TravelInformation))
g.add((RoutePlan, SKOS.prefLabel, Literal("route plan", lang="en")))
g.add((RoutePlan, SKOS.prefLabel, Literal("ruteplan", lang="nb")))
g.add((RoutePlan, SKOS.altLabel, Literal("stoppested", lang="nb")))
g.add((RoutePlan, SKOS.inScheme, OTDScheme))

TrafficInformation = URIRef(OTD + 'TrafficInformation')
g.add((TrafficInformation, RDF.type, SKOS.Concept))
g.add((TrafficInformation, SKOS.broader, Information))
g.add((TrafficInformation, SKOS.prefLabel, Literal("traffic information", lang="en")))
g.add((TrafficInformation, SKOS.prefLabel, Literal("trafikkinformasjon", lang="nb")))
g.add((TrafficInformation, SKOS.inScheme, OTDScheme))

Accident = URIRef(OTD + 'Accident')
g.add((Accident, RDF.type, SKOS.Concept))
g.add((Accident, SKOS.broader, TrafficInformation))
g.add((Accident, SKOS.inScheme, OTDScheme))
g.add((Accident, SKOS.prefLabel, Literal("accident", lang="en")))
g.add((Accident, SKOS.prefLabel, Literal("ulykke", lang="nb")))
g.add((Accident, SKOS.altLabel, Literal("skade", lang="nb")))
g.add((Accident, SKOS.inScheme, OTDScheme))

Incident = URIRef(OTD + 'Incident')
g.add((Incident, RDF.type, SKOS.Concept))
g.add((Incident, SKOS.broader, TrafficInformation))
g.add((Incident, SKOS.prefLabel, Literal("incident", lang="en")))
g.add((Incident, SKOS.prefLabel, Literal("episode", lang="nb")))
g.add((Incident, SKOS.altLabel, Literal("tilfelle", lang="nb")))
g.add((Incident, SKOS.inScheme, OTDScheme))

TrafficCondition = URIRef(OTD + 'TrafficCondition')
g.add((TrafficCondition, RDF.type, SKOS.Concept))
g.add((TrafficCondition, SKOS.broader, TrafficInformation))
g.add((TrafficCondition, SKOS.prefLabel, Literal("traffic condition", lang="en")))
g.add((TrafficCondition, SKOS.prefLabel, Literal("trafikkforhold", lang="nb")))
g.add((TrafficCondition, SKOS.inScheme, OTDScheme))

TransportNetworkCondition = URIRef(OTD + 'TransportNetworkCondition')
g.add((TransportNetworkCondition, RDF.type, SKOS.Concept))
g.add((TransportNetworkCondition, SKOS.broader, TrafficInformation))
g.add((TransportNetworkCondition, SKOS.prefLabel, Literal("transport network condition", lang="en")))
g.add((TransportNetworkCondition, SKOS.prefLabel, Literal("transportnettforhold", lang="nb")))
g.add((TransportNetworkCondition, SKOS.inScheme, OTDScheme))

TrackingInformation = URIRef(OTD + 'TrackingInformation')
g.add((TrackingInformation, RDF.type, SKOS.Concept))
g.add((TrackingInformation, SKOS.broader, TrafficInformation))
g.add((TrackingInformation, SKOS.prefLabel, Literal("tracking information", lang="en")))
g.add((TrackingInformation, SKOS.altLabel, Literal("tracking", lang="en")))
g.add((TrackingInformation, SKOS.prefLabel, Literal("sporing", lang="nb")))
g.add((TrackingInformation, SKOS.inScheme, OTDScheme))


Prognosis = URIRef(OTD + 'Prognosis')
g.add((Prognosis, RDF.type, SKOS.Concept))
g.add((Prognosis, SKOS.broader, Communication))
g.add((Prognosis, SKOS.prefLabel, Literal("prognosis", lang="en")))
g.add((Prognosis, SKOS.prefLabel, Literal("prognose", lang="nb")))
g.add((Prognosis, SKOS.inScheme, OTDScheme))

Forecast = URIRef(OTD + 'Forecast')
g.add((Forecast, RDF.type, SKOS.Concept))
g.add((Forecast, SKOS.broader, Prognosis))
g.add((Forecast, SKOS.prefLabel, Literal("forecast", lang="en")))
g.add((Forecast, SKOS.prefLabel, Literal("prediksjon", lang="nb")))
g.add((Forecast, SKOS.inScheme, OTDScheme))

WeatherForecast = URIRef(OTD + 'WeatherForecast')
g.add((WeatherForecast, RDF.type, SKOS.Concept))
g.add((WeatherForecast, SKOS.broader, Forecast))
g.add((WeatherForecast, SKOS.prefLabel, Literal("weather forecast", lang="en")))
g.add((WeatherForecast, SKOS.prefLabel, Literal("værmelding", lang="nb")))
g.add((WeatherForecast, SKOS.inScheme, OTDScheme))

Sign = URIRef(OTD + 'Sign')
g.add((Sign, RDF.type, SKOS.Concept))
g.add((Sign, SKOS.broader, Communication))
g.add((Sign, SKOS.prefLabel, Literal("sign", lang="en")))
g.add((Sign, SKOS.prefLabel, Literal("skilt", lang="nb")))
g.add((Sign, SKOS.inScheme, OTDScheme))

StreetSign = URIRef(OTD + 'StreetSign')
g.add((StreetSign, RDF.type, SKOS.Concept))
g.add((StreetSign, SKOS.broader, Sign))
g.add((StreetSign, SKOS.prefLabel, Literal("street sign", lang="en")))
g.add((StreetSign, SKOS.prefLabel, Literal("trafikkskilt", lang="nb")))
g.add((StreetSign, SKOS.inScheme, OTDScheme))

Group = URIRef(OTD + 'Group')
g.add((Group, RDF.type, SKOS.Concept))
g.add((Group, SKOS.broader, Abstraction))
g.add((Group, SKOS.prefLabel, Literal("group", lang="en")))
g.add((Group, SKOS.prefLabel, Literal("gruppe", lang="nb")))
g.add((Group, SKOS.inScheme, OTDScheme))

Collection = URIRef(OTD + 'Collection')
g.add((Collection, RDF.type, SKOS.Concept))
g.add((Collection, SKOS.broader, Group))
g.add((Collection, SKOS.prefLabel, Literal("collection", lang="en")))
g.add((Collection, SKOS.prefLabel, Literal("samling", lang="nb")))
g.add((Collection, SKOS.inScheme, OTDScheme))

Law = URIRef(OTD + 'Law')
g.add((Law, RDF.type, SKOS.Concept))
g.add((Law, SKOS.broader, Collection))
g.add((Law, SKOS.prefLabel, Literal("law", lang="en")))
g.add((Law, SKOS.prefLabel, Literal("lov", lang="nb")))
g.add((Law, SKOS.inScheme, OTDScheme))

Traffic = URIRef(OTD + 'Traffic')
g.add((Traffic, RDF.type, SKOS.Concept))
g.add((Traffic, SKOS.broader, Collection))
g.add((Traffic, SKOS.prefLabel, Literal("traffic", lang="en")))
g.add((Traffic, SKOS.prefLabel, Literal("trafikk", lang="nb")))
g.add((Traffic, SKOS.inScheme, OTDScheme))

Formation = URIRef(OTD + 'Formation')
g.add((Formation, RDF.type, SKOS.Concept))
g.add((Formation, SKOS.broader, Group))
g.add((Formation, SKOS.prefLabel, Literal("formation", lang="en")))
g.add((Formation, SKOS.prefLabel, Literal("formasjon", lang="nb")))
g.add((Formation, SKOS.inScheme, OTDScheme))

TrafficFlow = URIRef(OTD + 'TrafficFlow')
g.add((TrafficFlow, RDF.type, SKOS.Concept))
g.add((TrafficFlow, SKOS.broader, Formation))
g.add((TrafficFlow, SKOS.prefLabel, Literal("traffic flow", lang="en")))
g.add((TrafficFlow, SKOS.prefLabel, Literal("trafikkflyt", lang="nb")))
g.add((TrafficFlow, SKOS.inScheme, OTDScheme))

TrafficQueue = URIRef(OTD + 'TrafficQueue')
g.add((TrafficQueue, RDF.type, SKOS.Concept))
g.add((TrafficQueue, SKOS.broader, Formation))
g.add((TrafficQueue, SKOS.prefLabel, Literal("traffic queue", lang="en")))
g.add((TrafficQueue, SKOS.prefLabel, Literal("kø", lang="nb")))
g.add((TrafficQueue, SKOS.inScheme, OTDScheme))

Organization = URIRef(OTD + 'Organization')
g.add((Organization, RDF.type, SKOS.Concept))
g.add((Organization, SKOS.broader, Group))
g.add((Organization, SKOS.prefLabel, Literal("organization", lang="en")))
g.add((Organization, SKOS.prefLabel, Literal("organisasjon", lang="nb")))
g.add((Organization, SKOS.inScheme, OTDScheme))

Company = URIRef(OTD + 'Company')
g.add((Company, RDF.type, SKOS.Concept))
g.add((Company, SKOS.broader, Organization))
g.add((Company, SKOS.prefLabel, Literal("company", lang="en")))
g.add((Company, SKOS.prefLabel, Literal("selskap", lang="nb")))
g.add((Company, SKOS.inScheme, OTDScheme))

Object = URIRef(OTD + 'Object')
g.add((Object, RDF.type, SKOS.Concept))
g.add((Object, SKOS.broader, OTD.Entity))
g.add((Object, SKOS.prefLabel, Literal("object", lang="en")))
g.add((Object, SKOS.prefLabel, Literal("objekt", lang="nb")))
g.add((Object, SKOS.inScheme, OTDScheme))

Artifact = URIRef(OTD + 'Artifact')
g.add((Artifact, RDF.type, SKOS.Concept))
g.add((Artifact, SKOS.broader, Object))
g.add((Artifact, SKOS.prefLabel, Literal("artifact", lang="en")))
g.add((Artifact, SKOS.prefLabel, Literal("gjenstand", lang="nb")))
g.add((Artifact, SKOS.inScheme, OTDScheme))

Facility = URIRef(OTD + 'Facility')
g.add((Facility, RDF.type, SKOS.Concept))
g.add((Facility, SKOS.broader, Artifact))
g.add((Facility, SKOS.prefLabel, Literal("facility", lang="en")))
g.add((Facility, SKOS.prefLabel, Literal("anlegg", lang="nb")))
g.add((Facility, SKOS.inScheme, OTDScheme))

Airport = URIRef(OTD + 'Airport')
g.add((Airport, RDF.type, SKOS.Concept))
g.add((Airport, SKOS.broader, Facility))
g.add((Airport, SKOS.prefLabel, Literal("airport", lang="en")))
g.add((Airport, SKOS.prefLabel, Literal("flyplass", lang="nb")))
g.add((Airport, SKOS.inScheme, OTDScheme))

BusTerminal = URIRef(OTD + 'BusTerminal')
g.add((BusTerminal, RDF.type, SKOS.Concept))
g.add((BusTerminal, SKOS.broader, Terminal))
g.add((BusTerminal, SKOS.prefLabel, Literal("bus terminal", lang="en")))
g.add((BusTerminal, SKOS.altLabel, Literal("bus station", lang="en")))
g.add((BusTerminal, SKOS.altLabel, Literal("busstasjon", lang="nb")))
g.add((BusTerminal, SKOS.inScheme, OTDScheme))


Representation = URIRef(OTD + 'Representation')
g.add((Representation, RDF.type, SKOS.Concept))
g.add((Representation, SKOS.broader, Artifact))
g.add((Representation, SKOS.prefLabel, Literal("representation", lang="en")))
g.add((Representation, SKOS.prefLabel, Literal("representasjon", lang="nb")))
g.add((Representation, SKOS.inScheme, OTDScheme))

Structure = URIRef(OTD + 'Structure')
g.add((Structure, RDF.type, SKOS.Concept))
g.add((Structure, SKOS.broader, Artifact))
g.add((Structure, SKOS.prefLabel, Literal("structure", lang="en")))
g.add((Structure, SKOS.prefLabel, Literal("struktur", lang="nb")))
g.add((Structure, SKOS.inScheme, OTDScheme))

TransportNetwork = URIRef(OTD + 'TransportNetwork')
g.add((TransportNetwork, RDF.type, SKOS.Concept))
g.add((TransportNetwork, SKOS.broader, Structure))
g.add((TransportNetwork, SKOS.prefLabel, Literal("transport network", lang="en")))
g.add((TransportNetwork, SKOS.prefLabel, Literal("transportnettverk", lang="nb")))
g.add((TransportNetwork, SKOS.inScheme, OTDScheme))

Bridge = URIRef(OTD + 'Bridge')
g.add((Bridge, RDF.type, SKOS.Concept))
g.add((Bridge, SKOS.broader, Structure))
g.add((Bridge, SKOS.prefLabel, Literal("bridge", lang="en")))
g.add((Bridge, SKOS.prefLabel, Literal("bro", lang="nb")))
g.add((Bridge, SKOS.inScheme, OTDScheme))

Building = URIRef(OTD + 'Building')
g.add((Building, RDF.type, SKOS.Concept))
g.add((Building, SKOS.broader, Structure))
g.add((Building, SKOS.prefLabel, Literal("building", lang="en")))
g.add((Building, SKOS.prefLabel, Literal("bygning", lang="nb")))
g.add((Building, SKOS.inScheme, OTDScheme))

Garage = URIRef(OTD + 'Garage')
g.add((Garage, RDF.type, SKOS.Concept))
g.add((Garage, SKOS.broader, Building))
g.add((Garage, SKOS.prefLabel, Literal("garage", lang="en")))
g.add((Garage, SKOS.prefLabel, Literal("garasje", lang="nb")))
g.add((Garage, SKOS.inScheme, OTDScheme))

Booth = URIRef(OTD + 'Booth')
g.add((Booth, RDF.type, SKOS.Concept))
g.add((Booth, SKOS.broader, Structure))
g.add((Booth, SKOS.prefLabel, Literal("booth", lang="en")))
g.add((Booth, SKOS.prefLabel, Literal("skur", lang="nb")))
g.add((Booth, SKOS.inScheme, OTDScheme))

Toolbooth = URIRef(OTD + 'Toolbooth')
g.add((Toolbooth, RDF.type, SKOS.Concept))
g.add((Toolbooth, SKOS.broader, Booth))
g.add((Toolbooth, SKOS.prefLabel, Literal("tool booth", lang="en")))
g.add((Toolbooth, SKOS.prefLabel, Literal("bomstasjon", lang="nb")))
g.add((Toolbooth, SKOS.inScheme, OTDScheme))

Vehicle = URIRef(OTD + 'Vehicle')
g.add((Vehicle, RDF.type, SKOS.Concept))
g.add((Vehicle, SKOS.broader, Artifact))
g.add((Vehicle, SKOS.prefLabel, Literal("vehicle", lang="en")))
g.add((Vehicle, SKOS.prefLabel, Literal("kjøretøy", lang="nb")))
g.add((Vehicle, SKOS.inScheme, OTDScheme))

Way = URIRef(OTD + 'Way')
g.add((Way, RDF.type, SKOS.Concept))
g.add((Way, SKOS.broader, Artifact))
g.add((Way, SKOS.prefLabel, Literal("way", lang="en")))
g.add((Way, SKOS.prefLabel, Literal("strekning", lang="nb")))
g.add((Way, SKOS.inScheme, OTDScheme))

Road = URIRef(OTD + 'Road')
g.add((Road, RDF.type, SKOS.Concept))
g.add((Road, SKOS.broader, Way))
g.add((Road, SKOS.prefLabel, Literal("road", lang="en")))
g.add((Road, SKOS.prefLabel, Literal("vei", lang="nb")))
g.add((Road, SKOS.inScheme, OTDScheme))

Trail = URIRef(OTD + 'Trail')
g.add((Trail, RDF.type, SKOS.Concept))
g.add((Trail, SKOS.broader, Way))
g.add((Trail, SKOS.prefLabel, Literal("trail", lang="en")))
g.add((Trail, SKOS.prefLabel, Literal("sti", lang="nb")))
g.add((Trail, SKOS.inScheme, OTDScheme))

Detour = URIRef(OTD + 'Detour')
g.add((Detour, RDF.type, SKOS.Concept))
g.add((Detour, SKOS.broader, Road))
g.add((Detour, SKOS.prefLabel, Literal("detour", lang="en")))
g.add((Detour, SKOS.prefLabel, Literal("omvei", lang="nb")))
g.add((Detour, SKOS.altLabel, Literal("avstikker", lang="nb")))
g.add((Detour, SKOS.inScheme, OTDScheme))

Highway = URIRef(OTD + 'Highway')
g.add((Highway, RDF.type, SKOS.Concept))
g.add((Highway, SKOS.broader, Road))
g.add((Highway, SKOS.prefLabel, Literal("highway", lang="en")))
g.add((Highway, SKOS.prefLabel, Literal("hovedvei", lang="nb")))
g.add((Highway, SKOS.altLabel, Literal("motorvei", lang="nb")))
g.add((Highway, SKOS.inScheme, OTDScheme))

Roadway = URIRef(OTD + 'Roadway')
g.add((Roadway, RDF.type, SKOS.Concept))
g.add((Roadway, SKOS.broader, Road))
g.add((Roadway, SKOS.prefLabel, Literal("roadway", lang="en")))
g.add((Roadway, SKOS.prefLabel, Literal("landevei", lang="nb")))
g.add((Roadway, SKOS.inScheme, OTDScheme))

Speedway = URIRef(OTD + 'Speedway')
g.add((Speedway, RDF.type, SKOS.Concept))
g.add((Speedway, SKOS.broader, Road))
g.add((Speedway, SKOS.prefLabel, Literal("speedway", lang="en")))
g.add((Speedway, SKOS.prefLabel, Literal("speedway", lang="nb")))
g.add((Speedway, SKOS.inScheme, OTDScheme))

Tunnel = URIRef(OTD + 'Tunnel')
g.add((Tunnel, RDF.type, SKOS.Concept))
g.add((Tunnel, SKOS.broader, Way))
g.add((Tunnel, SKOS.prefLabel, Literal("tunnel", lang="en")))
g.add((Tunnel, SKOS.prefLabel, Literal("tunnel", lang="nb")))
g.add((Tunnel, SKOS.inScheme, OTDScheme))

Location = URIRef(OTD + 'Location')
g.add((Location, RDF.type, SKOS.Concept))
g.add((Location, SKOS.broader, Object))
g.add((Location, SKOS.prefLabel, Literal("location", lang="en")))
g.add((Location, SKOS.prefLabel, Literal("sted", lang="nb")))
g.add((Location, SKOS.altLabel, Literal("lokasjon", lang="nb")))
g.add((Location, SKOS.altLabel, Literal("beliggenhet", lang="nb")))
g.add((Location, SKOS.inScheme, OTDScheme))

POI = URIRef(OTD + 'POI')
g.add((POI, RDF.type, SKOS.Concept))
g.add((POI, SKOS.broader, Object))
g.add((POI, SKOS.inScheme, OTDScheme))
g.add((POI, SKOS.prefLabel, Literal("point of interest", lang="en")))
g.add((POI, SKOS.altLabel, Literal("poi", lang="en")))
g.add((POI, SKOS.prefLabel, Literal("interessepunkt", lang="nb")))

Region = URIRef(OTD + 'Region')
g.add((Region, RDF.type, SKOS.Concept))
g.add((Region, SKOS.broader, Location))
g.add((Region, SKOS.prefLabel, Literal("region", lang="en")))
g.add((Region, SKOS.prefLabel, Literal("region", lang="nb")))
g.add((Region, SKOS.inScheme, OTDScheme))

TopographicPoint = URIRef(OTD + 'TopographicPoint')
g.add((TopographicPoint, RDF.type, SKOS.Concept))
g.add((TopographicPoint, SKOS.broader, Location))
g.add((TopographicPoint, SKOS.prefLabel, Literal("topographic point", lang="en")))
g.add((TopographicPoint, SKOS.prefLabel, Literal("topografisk punkt", lang="nb")))
g.add((TopographicPoint, SKOS.inScheme, OTDScheme))

Junction = URIRef(OTD + 'Junction')
g.add((Junction, RDF.type, SKOS.Concept))
g.add((Junction, SKOS.broader, TopographicPoint))
g.add((Junction, SKOS.prefLabel, Literal("junction", lang="en")))
g.add((Junction, SKOS.prefLabel, Literal("veikryss", lang="nb")))
g.add((Junction, SKOS.inScheme, OTDScheme))

Interchange = URIRef(OTD + 'Interchange')
g.add((Interchange, RDF.type, SKOS.Concept))
g.add((Interchange, SKOS.broader, Junction))
g.add((Interchange, SKOS.prefLabel, Literal("interchange", lang="en")))
g.add((Interchange, SKOS.prefLabel, Literal("utveksling", lang="nb")))
g.add((Interchange, SKOS.inScheme, OTDScheme))

Intersection = URIRef(OTD + 'Intersection')
g.add((Intersection, RDF.type, SKOS.Concept))
g.add((Intersection, SKOS.broader, Junction))
g.add((Intersection, SKOS.prefLabel, Literal("intersection", lang="en")))
g.add((Intersection, SKOS.prefLabel, Literal("kryss", lang="nb")))
g.add((Intersection, SKOS.inScheme, OTDScheme))

RailwayJunction = URIRef(OTD + 'RailwayJunction')
g.add((RailwayJunction, RDF.type, SKOS.Concept))
g.add((RailwayJunction, SKOS.broader, Junction))
g.add((RailwayJunction, SKOS.prefLabel, Literal("railway junction", lang="en")))
g.add((RailwayJunction, SKOS.prefLabel, Literal("jernbaneovergang", lang="nb")))
g.add((RailwayJunction, SKOS.inScheme, OTDScheme))

TJunction = URIRef(OTD + 'TJunction')
g.add((TJunction, RDF.type, SKOS.Concept))
g.add((TJunction, SKOS.broader, Junction))
g.add((TJunction, SKOS.prefLabel, Literal("t junction", lang="en")))
g.add((TJunction, SKOS.altLabel, Literal("3-way junction", lang="en")))
g.add((TJunction, SKOS.prefLabel, Literal("3-veis kryss", lang="nb")))
g.add((TJunction, SKOS.inScheme, OTDScheme))

TrafficCircle = URIRef(OTD + 'TrafficCircle')
g.add((TrafficCircle, RDF.type, SKOS.Concept))
g.add((TrafficCircle, SKOS.broader, Junction))
g.add((TrafficCircle, SKOS.prefLabel, Literal("traffic circle", lang="en")))
g.add((TrafficCircle, SKOS.prefLabel, Literal("rundkjøring", lang="nb")))
g.add((TrafficCircle, SKOS.inScheme, OTDScheme))

ServiceArea = URIRef(OTD + 'ServiceArea')
g.add((ServiceArea, RDF.type, SKOS.Concept))
g.add((ServiceArea, SKOS.broader, TopographicPoint))
g.add((ServiceArea, SKOS.prefLabel, Literal("service area", lang="en")))
g.add((ServiceArea, SKOS.prefLabel, Literal("serviceområde", lang="nb")))
g.add((ServiceArea, SKOS.inScheme, OTDScheme))

TransferNode = URIRef(OTD + 'TransferNode')
g.add((TransferNode, RDF.type, SKOS.Concept))
g.add((TransferNode, SKOS.broader, TopographicPoint))
g.add((TransferNode, SKOS.prefLabel, Literal("transfer node", lang="en")))
g.add((TransferNode, SKOS.altLabel, Literal("transfer point", lang="en")))
g.add((TransferNode, SKOS.prefLabel, Literal("overføringspunkt", lang="nb")))
g.add((TransferNode, SKOS.inScheme, OTDScheme))

Terminal = URIRef(OTD + 'Terminal')
g.add((Terminal, RDF.type, SKOS.Concept))
g.add((Terminal, SKOS.broader, TransferNode))
g.add((Terminal, SKOS.prefLabel, Literal("terminal", lang="en")))
g.add((Terminal, SKOS.prefLabel, Literal("terminal", lang="nb")))
g.add((Terminal, SKOS.inScheme, OTDScheme))


Parking = URIRef(OTD + 'Parking')
g.add((Parking, RDF.type, SKOS.Concept))
g.add((Parking, SKOS.broader, TransferNode))
g.add((Parking, SKOS.prefLabel, Literal("parking", lang="en")))
g.add((Parking, SKOS.prefLabel, Literal("parkering", lang="nb")))
g.add((Parking, SKOS.inScheme, OTDScheme))

Park_and_Ride = URIRef(OTD + 'Park_and_Ride')
g.add((Park_and_Ride, RDF.type, SKOS.Concept))
g.add((Park_and_Ride, SKOS.broader, TransferNode))
g.add((Park_and_Ride, SKOS.prefLabel, Literal("park and ride", lang="en")))
g.add((Park_and_Ride, SKOS.altLabel, Literal("coaching inn", lang="en")))
g.add((Park_and_Ride, SKOS.prefLabel, Literal("skysstasjon", lang="nb")))
g.add((Park_and_Ride, SKOS.inScheme, OTDScheme))

Zone = URIRef(OTD + 'Zone')
g.add((Zone, RDF.type, SKOS.Concept))
g.add((Zone, SKOS.broader, TopographicPoint))
g.add((Zone, SKOS.prefLabel, Literal("zone", lang="en")))
g.add((Zone, SKOS.prefLabel, Literal("sone", lang="nb")))
g.add((Zone, SKOS.inScheme, OTDScheme))

Organism = URIRef(OTD + 'Organism')
g.add((Organism, RDF.type, SKOS.Concept))
g.add((Organism, SKOS.broader, Object))
g.add((Organism, SKOS.prefLabel, Literal("organism", lang="en")))
g.add((Organism, SKOS.prefLabel, Literal("organisme", lang="nb")))
g.add((Organism, SKOS.inScheme, OTDScheme))

Process = URIRef(OTD + 'Process')
g.add((Process, RDF.type, SKOS.Concept))
g.add((Process, SKOS.broader, Entity))
g.add((Process, SKOS.prefLabel, Literal("process", lang="en")))
g.add((Process, SKOS.prefLabel, Literal("prosess", lang="nb")))
g.add((Process, SKOS.inScheme, OTDScheme))

TransportService = URIRef(OTD + 'TransportService')
g.add((TransportService, RDF.type, SKOS.Concept))
g.add((TransportService, SKOS.broader, Process))
g.add((TransportService, SKOS.prefLabel, Literal("transport service", lang="en")))
g.add((TransportService, SKOS.prefLabel, Literal("transporttjeneste", lang="nb")))
g.add((TransportService, SKOS.inScheme, OTDScheme))

CityBike = URIRef(OTD + 'CityBike')
g.add((CityBike, RDF.type, SKOS.Concept))
g.add((CityBike, SKOS.broader, OTD.TransportService))
g.add((CityBike, SKOS.prefLabel, Literal("city bike", lang="en")))
g.add((CityBike, SKOS.prefLabel, Literal("bysykkel", lang="nb")))
g.add((CityBike, SKOS.inScheme, OTD.OTDScheme))

Taxi = URIRef(OTD + 'Taxi')
g.add((Taxi, RDF.type, SKOS.Concept))
g.add((Taxi, SKOS.broader, OTD.TransportService))
g.add((Taxi, SKOS.prefLabel, Literal("taxi", lang="en")))
g.add((Taxi, SKOS.altLabel, Literal("drosje", lang="en")))
g.add((Taxi, SKOS.prefLabel, Literal("taxi", lang="nb")))
g.add((Taxi, SKOS.inScheme, OTD.OTDScheme))

CarSharing = URIRef(OTD + 'CarSharing')
g.add((CarSharing, RDF.type, SKOS.Concept))
g.add((CarSharing, SKOS.broader, TransportService))
g.add((CarSharing, SKOS.prefLabel, Literal("carsharing", lang="en")))
g.add((CarSharing, SKOS.altLabel, Literal("car pooling", lang="en")))
g.add((CarSharing, SKOS.prefLabel, Literal("bildeling", lang="nb")))
g.add((CarSharing, SKOS.inScheme, OTDScheme))

Event = URIRef(OTD + 'Event')
g.add((Event, RDF.type, SKOS.Concept))
g.add((Event, SKOS.broader, Process))
g.add((Event, SKOS.prefLabel, Literal("event", lang="en")))
g.add((Event, SKOS.prefLabel, Literal("begivenhet", lang="nb")))
g.add((Event, SKOS.inScheme, OTDScheme))

Action = URIRef(OTD + 'Action')
g.add((Action, RDF.type, SKOS.Concept))
g.add((Action, SKOS.broader, Event))
g.add((Action, SKOS.prefLabel, Literal("action", lang="en")))
g.add((Action, SKOS.prefLabel, Literal("handling", lang="nb")))
g.add((Action, SKOS.inScheme, OTDScheme))

Activity = URIRef(OTD + 'Activity')
g.add((Activity, RDF.type, SKOS.Concept))
g.add((Activity, SKOS.broader, Event))
g.add((Activity, SKOS.prefLabel, Literal("activity", lang="en")))
g.add((Activity, SKOS.prefLabel, Literal("aktivitet", lang="nb")))
g.add((Activity, SKOS.inScheme, OTDScheme))

Happening = URIRef(OTD + 'Happening')
g.add((Happening, RDF.type, SKOS.Concept))
g.add((Happening, SKOS.broader, Event))
g.add((Happening, SKOS.prefLabel, Literal("happening", lang="en")))
g.add((Happening, SKOS.prefLabel, Literal("hendelse", lang="nb")))
g.add((Happening, SKOS.inScheme, OTDScheme))

#Accident = URIRef(OTD + 'Accident')
#g.add((Accident, RDF.type, SKOS.Concept))
#g.add((Accident, SKOS.broader, Happening))
#g.add((Accident, SKOS.prefLabel, Literal("accident", lang="en")))
#g.add((Accident, SKOS.prefLabel, Literal("ulykke", lang="nb")))
#g.add((Accident, SKOS.inScheme, OTDScheme))

#Incident = URIRef(OTD + 'Incident')
#g.add((Incident, RDF.type, SKOS.Concept))
#g.add((Incident, SKOS.broader, Happening))
#g.add((Incident, SKOS.prefLabel, Literal("incident", lang="en")))
#g.add((Incident, SKOS.prefLabel, Literal("episode", lang="nb")))
#g.add((Incident, SKOS.inScheme, OTDScheme))

#TrafficCondition = URIRef(OTD + 'TrafficCondition')
#g.add((TrafficCondition, RDF.type, SKOS.Concept))
#g.add((TrafficCondition, SKOS.broader, Happening))
#g.add((TrafficCondition, SKOS.prefLabel, Literal("traffic condition", lang="en")))
#g.add((TrafficCondition, SKOS.prefLabel, Literal("trafikkforhold", lang="nb")))
#g.add((TrafficCondition, SKOS.inScheme, OTDScheme))

Phenomenon = URIRef(OTD + 'Phenomenon')
g.add((Phenomenon, RDF.type, SKOS.Concept))
g.add((Phenomenon, SKOS.broader, Process))
g.add((Phenomenon, SKOS.prefLabel, Literal("phenomenon", lang="en")))
g.add((Phenomenon, SKOS.prefLabel, Literal("fenomen", lang="nb")))
g.add((Phenomenon, SKOS.inScheme, OTDScheme))

#TransportNetworkCondition = URIRef(OTD + 'TransportNetworkCondition')
#g.add((TransportNetworkCondition, RDF.type, SKOS.Concept))
#g.add((TransportNetworkCondition, SKOS.broader, Phenomenon))
#g.add((TransportNetworkCondition, SKOS.prefLabel, Literal("transport network condition", lang="en")))
#g.add((TransportNetworkCondition, SKOS.prefLabel, Literal("transportnettilstand", lang="nb")))
#g.add((TransportNetworkCondition, SKOS.inScheme, OTDScheme))

WeatherCondition = URIRef(OTD + 'WeatherCondition')
g.add((WeatherCondition, RDF.type, SKOS.Concept))
g.add((WeatherCondition, SKOS.broader, Phenomenon))
g.add((WeatherCondition, SKOS.prefLabel, Literal("weather condition", lang="en")))
g.add((WeatherCondition, SKOS.prefLabel, Literal("værforhold", lang="nb")))
g.add((WeatherCondition, SKOS.inScheme, OTDScheme))

Release = URIRef(OTD + 'Release')
g.add((Release, RDF.type, SKOS.Concept))
g.add((Release, SKOS.broader, Process))
g.add((Release, SKOS.prefLabel, Literal("release", lang="en")))
g.add((Release, SKOS.prefLabel, Literal("frigjøre", lang="nb")))
g.add((Release, SKOS.definition, Literal("the production and discharge of something, especially gas or radiation.", lang="en")))
g.add((Release, SKOS.inScheme, OTDScheme))

Line = URIRef(OTD + 'Line')
g.add((Line, RDF.type, SKOS.Concept))
g.add((Line, SKOS.broader, OTD.Location))
g.add((Line, SKOS.prefLabel, Literal("line", lang="en")))
g.add((Line, SKOS.prefLabel, Literal("linje", lang="nb")))
g.add((Line, SKOS.inScheme, OTD.OTDScheme))

Route = URIRef(OTD + 'Route')
g.add((Route, RDF.type, SKOS.Concept))
g.add((Route, SKOS.broader, OTD.Line))
g.add((Route, SKOS.prefLabel, Literal("route", lang="en")))
g.add((Route, SKOS.prefLabel, Literal("rute", lang="nb")))
g.add((Route, SKOS.altLabel, Literal("trasse", lang="nb")))
g.add((Route, SKOS.inScheme, OTD.OTDScheme))

Schedule = URIRef(OTD + 'Schedule')
g.add((Schedule, RDF.type, SKOS.Concept))
g.add((Schedule, SKOS.broader, OTD.TravelInformation))
g.add((Schedule, SKOS.prefLabel, Literal("schedule", lang="en")))
g.add((Schedule, SKOS.prefLabel, Literal("agenda", lang="nb")))
g.add((Schedule, SKOS.inScheme, OTD.OTDScheme))

Timetable = URIRef(OTD + 'Timetable')
g.add((Timetable, RDF.type, SKOS.Concept))
g.add((Timetable, SKOS.broader, OTD.Schedule))
g.add((Timetable, SKOS.prefLabel, Literal("timetable", lang="en")))
g.add((Timetable, SKOS.prefLabel, Literal("rutetabell", lang="nb")))
g.add((Timetable, SKOS.inScheme, OTD.OTDScheme))

PublicTransport = URIRef(OTD + 'PublicTransport')
g.add((PublicTransport, RDF.type, SKOS.Concept))
g.add((PublicTransport, SKOS.broader, OTD.Vehicle))
g.add((PublicTransport, SKOS.prefLabel, Literal("public transport", lang="en")))
g.add((PublicTransport, SKOS.prefLabel, Literal("offentlig transport", lang="nb")))
g.add((PublicTransport, SKOS.altLabel, Literal("transport", lang="nb")))
g.add((PublicTransport, SKOS.inScheme, OTD.OTDScheme))

Train = URIRef(OTD + 'Train')
g.add((Train, RDF.type, SKOS.Concept))
g.add((Train, SKOS.broader, OTD.PublicTransport))
g.add((Train, SKOS.prefLabel, Literal("train", lang="en")))
g.add((Train, SKOS.prefLabel, Literal("tog", lang="nb")))
g.add((Train, SKOS.altLabel, Literal("jernbane", lang="nb")))
g.add((Train, SKOS.inScheme, OTD.OTDScheme))

Bus = URIRef(OTD + 'Bus')
g.add((Bus, RDF.type, SKOS.Concept))
g.add((Bus, SKOS.broader, OTD.PublicTransport))
g.add((Bus, SKOS.prefLabel, Literal("bus", lang="en")))
g.add((Bus, SKOS.prefLabel, Literal("buss", lang="nb")))
g.add((Bus, SKOS.inScheme, OTD.OTDScheme))

Boat = URIRef(OTD + 'Boat')
g.add((Boat, RDF.type, SKOS.Concept))
g.add((Boat, SKOS.broader, OTD.PublicTransport))
g.add((Boat, SKOS.prefLabel, Literal("boat", lang="en")))
g.add((Boat, SKOS.prefLabel, Literal(u"båt", lang="nb")))
g.add((Boat, SKOS.inScheme, OTD.OTDScheme))

PersonalTransport = URIRef(OTD + 'PersonalTransport')
g.add((PersonalTransport, RDF.type, SKOS.Concept))
g.add((PersonalTransport, SKOS.broader, OTD.Vehicle))
g.add((PersonalTransport, SKOS.prefLabel, Literal("personal transport", lang="en")))
g.add((PersonalTransport, SKOS.prefLabel, Literal("persolig transport", lang="nb")))
g.add((PersonalTransport, SKOS.altLabel, Literal("skyss", lang="nb")))
g.add((PersonalTransport, SKOS.inScheme, OTD.OTDScheme))

Bicycle = URIRef(OTD + 'Bicycle')
g.add((Bicycle, RDF.type, SKOS.Concept))
g.add((Bicycle, SKOS.broader, OTD.PersonalTransport))
g.add((Bicycle, SKOS.prefLabel, Literal("bicycle", lang="en")))
g.add((Bicycle, SKOS.altLabel, Literal("bike", lang="en")))
g.add((Bicycle, SKOS.prefLabel, Literal("sykkel", lang="nb")))
g.add((Bicycle, SKOS.inScheme, OTD.OTDScheme))

Plan = URIRef(OTD + 'Plan')
g.add((Plan, RDF.type, SKOS.Concept))
g.add((Plan, SKOS.broader, OTD.Representation))
g.add((Plan, SKOS.prefLabel, Literal("plan", lang="en")))
g.add((Plan, SKOS.prefLabel, Literal(u"plan", lang="nb")))
g.add((Plan, SKOS.inScheme, OTD.OTDScheme))

RoutePlan = URIRef(OTD + 'RoutePlan')
g.add((RoutePlan, RDF.type, SKOS.Concept))
g.add((RoutePlan, SKOS.broader, OTD.Plan))
g.add((RoutePlan, SKOS.prefLabel, Literal("route plan", lang="en")))
g.add((RoutePlan, SKOS.prefLabel, Literal(u"ruteplan", lang="nb")))
g.add((RoutePlan, SKOS.inScheme, OTD.OTDScheme))

BusStop = URIRef(OTD + 'BusStop')
g.add((BusStop, RDF.type, SKOS.Concept))
g.add((BusStop, SKOS.broader, OTD.TransferNode))
g.add((BusStop, SKOS.prefLabel, Literal("bus stop", lang="en")))
g.add((BusStop, SKOS.prefLabel, Literal(u"busstopp", lang="nb")))
g.add((BusStop, SKOS.inScheme, OTD.OTDScheme))

Station = URIRef(OTD + 'Station')
g.add((Station, RDF.type, SKOS.Concept))
g.add((Station, SKOS.broader, OTD.TransferNode))
g.add((Station, SKOS.prefLabel, Literal("station", lang="en")))
g.add((Station, SKOS.prefLabel, Literal(u"stasjon", lang="nb")))
g.add((Station, SKOS.inScheme, OTD.OTDScheme))

ChargingStation = URIRef(OTD + 'ChargingStation')
g.add((ChargingStation, RDF.type, SKOS.Concept))
g.add((ChargingStation, SKOS.broader, OTD.Station))
g.add((ChargingStation, SKOS.prefLabel, Literal("charging station", lang="en")))
g.add((ChargingStation, SKOS.altLabel, Literal("charging", lang="en")))
#g.add((ChargingStation, SKOS.altLabel, Literal("station", lang="en")))
g.add((ChargingStation, SKOS.altLabel, Literal("charging service", lang="en")))
g.add((ChargingStation, SKOS.altLabel, Literal("filling station", lang="en")))
g.add((ChargingStation, SKOS.prefLabel, Literal(u"ladestasjon", lang="nb")))
g.add((ChargingStation, SKOS.altLabel, Literal(u"lading", lang="nb")))
g.add((ChargingStation, SKOS.inScheme, OTD.OTDScheme))

g.serialize(destination="otd-ontology-v6.owl", format="xml")
